In [16]:
import requests
from html import unescape
import re
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

In [17]:
url ='https://www.bbc.com/sitemaps/https-index-com-archive.xml'

In [18]:
res = requests.get(url)
if res.status_code != 200:
    print('ohho')

soup = BeautifulSoup(res.text, 'xml')

In [19]:
sitemap_tags = soup.find_all('sitemap')

In [20]:
# sitemap_tags_last_ten_days = []

# for stitemap_tag in sitemap_tags:
#     date_str = stitemap_tag.lastmod.text
#     date_obj = datetime.fromisoformat(date_str[:-1])

#     # Check if the date is within the last 10 days
#     today = datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0)
#     if today - timedelta(days=10) <= date_obj:
#         sitemap_tags_last_ten_days.append(stitemap_tag)
#     else:
#         continue
    

In [21]:
# sitemap_tags = sitemap_tags_last_ten_days

In [22]:
sitemaps = []
for sitemap in sitemap_tags:
    loc = sitemap.find('loc')
    sitemaps.append(loc.text)

In [26]:
urls = []
regex_1 = r"^https://www.bbc.com/news/.*$"
regex_2 = r"^https://www.bbc.com/sport/.*$"

today = datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0)
threshold = 10
threshold_date = today - timedelta(days=threshold)

for sitemap in sitemaps:
        try:
                response = requests.get(sitemap)
                soup = BeautifulSoup(response.text, 'xml')
                url_tags = soup.find_all('url')
                for url in url_tags:
                        try:
                                date_str = url.lastmod.text
                                date_obj = datetime.fromisoformat(date_str[:-1])
                                if threshold_date <= date_obj:
                                        loc = url.find('loc')
                                        if re.match(regex_1, loc.text) or re.match(regex_2, loc.text):
                                                urls.append(loc.text)
                                else:
                                        continue
                        except AttributeError as e:
                                continue

        except requests.exceptions.Timeout as ee :
                continue


In [27]:
len(set(urls))

5194

In [28]:
def url_to_text(url):
    try:
        response = requests.get(url)
        if response.status_code != 200:
            return None
        
        soup = BeautifulSoup(response.text, 'html.parser')
        full_article = ''
        text = []
        heading = soup.select('h1')[0].text.strip()
        text.append(heading)
        
        for p in soup.select('p'):
            text.append(unescape(p.text))
        full_article = ' '.join(text)
        
        return (url, full_article) 
    except requests.exceptions.Timeout as ee :
        return None

In [29]:
import psycopg2

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="SearchEngine",
    user="postgres",
    password="2580"
)

# Open a cursor to perform database operations
cur = conn.cursor()


for url in urls:
    try:
        url, text = url_to_text(url)
        data = (url, text)
        sql = sql = "INSERT INTO sites (url, content) VALUES (%s, %s)"
        cur.execute(sql, data)
        conn.commit()
    except TypeError as ee :
        continue
